In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('../py'))
from Secrets import ReadSecrets

from pandas_datareader import data as pdr

import pandas as pd

import yfinance as yf
yf.pdr_override()
from datetime import datetime as calendar
import numpy as np
import chart_studio.plotly as plty

secrets = ReadSecrets().read('../secrets.json')
plty.sign_in(secrets.secrets['plotly']['username'], secrets.secrets['plotly']['apiKey'])

import plotly.graph_objs as gobjs

/home/andre/Desktop/pythonTradingAlgos/tradingalgos/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
def get_history(tickers, start_date, end_date):
    def data(ticker):
        return (pdr.get_data_yahoo(ticker, start=start_date, end=end_date))
        
    tickers_data = map(data, tickers)
    
    return pd.concat(tickers_data, keys=tickers, names = ['Ticker', 'Date'])

In [4]:
tickers = ['AAPL', 'MSFT', '^GSPC']
start_date = calendar(2016, 1 ,1)
end_date = calendar(2018, 12, 31)

data = get_history(tickers, start_date, end_date)
data = data.reset_index()
data = data.set_index(['Date', 'Ticker']).sort_index()
close = data['Close']

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [5]:
display(data)


Open         High          Low        Close  \
Date       Ticker                                                       
2016-01-04 AAPL     102.610001   105.370003   102.000000   105.349998   
           MSFT      54.320000    54.799999    53.389999    54.799999   
           ^GSPC   2038.199951  2038.199951  1989.680054  2012.660034   
2016-01-05 AAPL     105.750000   105.849998   102.410004   102.709999   
           MSFT      54.930000    55.389999    54.540001    55.049999   
...                        ...          ...          ...          ...   
2018-12-27 MSFT      99.300003   101.190002    96.400002   101.180000   
           ^GSPC   2442.500000  2489.100098  2397.939941  2488.830078   
2018-12-28 AAPL     157.500000   158.520004   154.550003   156.229996   
           MSFT     102.089996   102.410004    99.519997   100.389999   
           ^GSPC   2498.770020  2520.270020  2472.889893  2485.739990   

                     Adj Close      Volume  
Date       Ticker                           
2016-01-04 AAPL      98.213585    67649400  
           MSFT      50.536495    53778000  
           ^GSPC   2012.660034  4304880000  
2016-01-05 AAPL      95.752419    55791000  
           MSFT      50.767044    34079700  
...                        ...         ...  
2018-12-27 MSFT      99.705742    49498500  
           ^GSPC   2488.830078  4096610000  
2018-12-28 AAPL     153.552979    42291400  
           MSFT      98.927246    38196300  
           ^GSPC   2485.739990  3702620000  

[2259 rows x 6 columns]

In [6]:
display(close)

Date        Ticker
2016-01-04  AAPL       105.349998
            MSFT        54.799999
            ^GSPC     2012.660034
2016-01-05  AAPL       102.709999
            MSFT        55.049999
                         ...     
2018-12-27  MSFT       101.180000
            ^GSPC     2488.830078
2018-12-28  AAPL       156.229996
            MSFT       100.389999
            ^GSPC     2485.739990
Name: Close, Length: 2259, dtype: float64

In [7]:
weekdays = pd.date_range(start=start_date, end=end_date, freq='B') # all weekdays in the interval
close = close.reindex(pd.MultiIndex.from_product([weekdays, tickers], names=['Date', 'Ticker']), fill_value=np.NaN)

In [8]:
close.head(10)

Date        Ticker
2016-01-01  AAPL              NaN
            MSFT              NaN
            ^GSPC             NaN
2016-01-04  AAPL       105.349998
            MSFT        54.799999
            ^GSPC     2012.660034
2016-01-05  AAPL       102.709999
            MSFT        55.049999
            ^GSPC     2016.709961
2016-01-06  AAPL       100.699997
Name: Close, dtype: float64

In [9]:
close = close.reset_index().pivot(index='Date', columns='Ticker', values='Close')